In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 模型参数
- 数据流图中的可训练参数几乎就是权重值，偏置值
- 存储：checkpoint <变量， 张量值>，ckpt文件（P92）

### 创建，初始化，更新

#### 1 创建
- 常见的生成方式(P93)
    - 正态分布，截尾正态分布，均匀分布，多项式分布，伽马分布等
    - random_shuffle：按维度重洗
    - zero_like()之类的，利用某种生成规则
    - constant，常量
- 利用已经创建的量
    - Tensor.initialized_value(),可以放缩等处理作为别的变量

In [ ]:
#使用initialized_value()方法，注意不能直接把变量实例传进去
W = tf.Variable(tf.random_normal(shape=(1, 4), steddev=0.35), name='W')
W2 = tf.Variable(W.initialized_value()*2, name='W2')

In [ ]:
W = tf.Variable(tf.random_normal(shape=(1, 4), stddev=0.35), name='W')
    W2 = tf.Variable(W.initialized_value(), name='W2')

    initer = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(initer)
        writer = tf.summary.FileWriter('logs_1/', sess.graph)

    得到的图如下，initial_value变为一个数据生成器
![](imgs/8.png)

#### 2 初始化
- 初始化全局变量 tf.global_variable_initializer(), 在分布系统中能够实现跨设备全局初始化。实际上也是调用下一个方法(P96)
- 部分变量 tf.variable_initializer(var_list, name),这里var_list是同类变量集合列表，5类(P96), 在创建变量时可以直接通过collection指定，collection的默认为GraphKeys.GLOBAL_VARIABLES,即全局变量
    ```python
        W = tf.Variable(W, collections=[GraphKeys.GLOBAL_VARIABLES])
    ```
    - tensorflow提供了5种不同的变量集合类型，通过各自名称的小写表示可以得到对应集合，比如tf.global_variables()
- 变量初始化的调试变量方法(P96)
    - 是否初始化：
    - 未初始化变量集合列表获取
    - 初始化断言

#### 3 模型参数更新
- 模型参数更新实际上是对变量的重新赋值
- tf.assign(W, 1.) 注意：在数据流图不变下对变量进行更新，比如梯度的更新，而不是直接用‘=’赋值

### 模型保存(P98)

## 模块化的网络搭建
- 使用变量作用域
    - [tf.get_variable(name, shape, initializer)](https://tensorflow.google.cn/api_docs/python/tf/get_variable)：创建或获取指定名称的变量
        - 注意一下有一个不常用但应该用的reguglarizer参数
        - initializer:
            - [tf.constant_initializer](https://tensorflow.google.cn/api_docs/python/tf/constant_initializer)
            - tf.random_uniform_initializer
            - tf.random_normal_initializer
    - [tf.variable_scope]():用于管理变量子空间

In [ ]:
#(添加)一个卷积层的函数
def conv_relu(inputs, params):
    kernel_shape, bias_shape, strides, padding = some_function(params)
    weights = tf.get_variable(name='weights', shape=kernel_shape, initializer=tf.random_normal_initializer())
    biases = tf.get_variable(name='biases', shape=bias_shape, initizlizer=tf.constant_initializer(0.))
    conv = tf.nn.conv2d(inputs, weights, strides, padding)
    return tf.nn.relu(conv + biases)

##### 这里如果第二次调用model()会产生ValueError异常，用当加上reuse参数(P102)

In [ ]:
def model(inputs, params):
    #with 上下文定义的变量，会加上variable_scope()中定义的前缀
    with tf.variable_scope('conv1', reuse=True):#conv1/weight
        A1 = conv_relu(inputs, params)
    with tf.variable_scope('conv2', reuse=True):
        A2 = conv_relu(A1, params)

##### 关于变量名称，嵌套命名

In [ ]:
with tf.variable_scope('foo'):
    with tf.variable_scope('bar'):
        v = tf.get_variable("v", [1])
assert v.name == "foo/bar/v:0"

##### 关于初始化方法，嵌套关系

In [ ]:
with tf.variable_scope('foo', initializer=tf.constant_initializer(.4), reuse=True):
    v = tf.get_variable('v', shape=[1])#继承前面的初始化方法
    w = tf.get_variable('w', [1], initializer=tf.random_normal_initializer())#使用自己的初始化方法
    with tf.variable_scope('bar', initializer=tf.random_uniform_initializer()):
        c = tf.get_variable('c', [1])#如果自己没有定义，就使用最接近的初始化方法

#### 变量名子空间
- tf.variable_scope()
- tf.name_scope()
- tf.get_variable()是创建或获取指定名称的变量，name_scope()不会作为命名的一部分